# Movie Budget Risk Analysis

## Business Understanding

- Stakeholders: a company wanting to enter the orginail video content creation space by opening a new movie studio.


- Task: Identify which types of films are currently the most successful and summarize these findings for stakeholders.

## Data Understanding

The data explored for this task came from two sources:

- [The Numbers](https://www.the-numbers.com/): A film data website which provided production budgeting information and worldwide gross revenue estimations for the films we analyzed. This website is operated by [OpusData](https://www.opusdata.com/home.php), and it allows access to all the financial film data via its web-based API.
    - The data from this website was stored in a csv file - '../unzippedData/tn.movie_budgets.csv' - that is linked below and included in our repository for reference.


- [IMDb](https://www.imdb.com/): A website that functions to gather and present information about the success of films based on metrics like ratings, while also contributing categorical metrics such as attributing directors to films and connecting principal actors to their work. We utilized this website to categorize films into their appropriate genres as well as to identify the directors that worked on each of the films.
    - The data from IMDb is stored as a SQL database and can also be found in our repository for refernce. We access it below using the following path: '../unzippedData/im.db'

## Data Preparation

- To begin this project, we will import all necessary python libraries that will be utilized throughout the notebook. Then we will access our data via connecting to the data files stored in our repositroy. 

In [1]:
# Importing python libraries
import pandas as pd
import sqlite3
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Viewing the data sources in the repository stored in the file 'unzippedData'
! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [3]:
# Reading the csv file containing The Numbers movie data and displaying the columns for an initial view of the data.
MovieBudget = pd.read_csv('../unzippedData/tn.movie_budgets.csv')
MovieBudget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [4]:
# Connecting to IMDb SQL database.
conn = sqlite3.connect('../unzippedData/im.db')

### Displaying the Entity Relational Diagram of all the tables and their respective columns in the IMDb database.
- This visual allows users to recognize how each of the tables are connected via designated foreign keys.
![ERD Diagram of IMDb SQL database](../images/movie_data_erd.jpeg)

### Data Cleaning
- Through the creation of the various functions detialed below, we were able to scrub the data from our various sources. Cleaning the data allowed us to run aggregating functions in the later stages of our data analysis.

In [5]:
# Function for cleaning numbers column, removing unnecessary characters and casting the column as integers.

def clean_number_series(Series):
    Series = Series.str.replace(",", "")
    Series = Series.str.replace("$", "")
    Series = Series.astype(int)
    return Series

# Function for cleaning column headers.

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")
    
# Function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

### Applying functions to The Numbers dataset.

In [6]:
# Running the 'MovieBudget' variable through the functions.
clean_column_headers(MovieBudget)
MovieBudget['Worldwide Gross'] = clean_number_series(MovieBudget['Worldwide Gross'])
MovieBudget['Domestic Gross'] = clean_number_series(MovieBudget['Domestic Gross'])
MovieBudget['Production Budget'] = clean_number_series(MovieBudget['Production Budget'])

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/2749743923.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Series = Series.str.replace("$", "")


## Data Analysis

### Analysis Overview:
1. Identify which metric best measures the "success" of a particular film.

2. Explore the different options available to categorize films (using genre classification or decade, etc). 

3. Developing a strategy to present this data to the stakeholders. 

        - As the project unfolded, additional data cleaning and preparation occured and will be detailed below.


#### 1. Measuring Film Success
Using The Numbers dataset, we identified the columns 'Worldwide Gross' revenue and 'Production Budget' as the key metrics that we could use in our analysis. Each record in the dataset corresponds to a unique film and it includes the date of release for that film. 

First we created the column 'Profitability' to gain insight on exaclty how much money was earned by each film. The equation for this column subtracts the 'Production Budget' or the cost to create the movie from the revenues earned which is stored in the 'Worldwide Gross' column for each film.

In [7]:
# Adding 'Profitability' column to The Numbers dataset.
MovieBudget['Profitability'] = MovieBudget['Worldwide Gross'] - MovieBudget['Production Budget']

We determined that we could not rely on this metric alone, as this film data spans many decades and there is no indication that the financial figures provided have been adjusted for inflation.

Next we decided to engineer a ratio between the revenues earned worldwide and the cost to create the movie. 

In [8]:
# Calculating the ratio bewteen revenues earned an the cost to make the movie.
MovieBudget['ROI'] = MovieBudget['Worldwide Gross'] / MovieBudget['Production Budget']

This engineered ratio allows us to ignore additional (and potentially inaccurate) inflation adjustments over time, as each film incurred costs and revenues in it's own time, so the 'Worldwide Gross' and 'Production Budget' values may be combined and analysed without additional scaling.

In [9]:
# Checking to see if the columns were properly added.
MovieBudget

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,6.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696
...,...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,0.000000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495,40.082500
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662,0.267600
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,0.000000


Our next step was to clean the Release Date column. We determined that the month and day information was too granular for our analysis at this stage, so we isolated the release year associated with each film.

In [10]:
# Redefining the 'Release Date' column to just display the release year and then checking how many films 
#    are documented for each year.
MovieBudget_Prepared = MovieBudget.copy()
MovieBudget_Prepared['Release Date'] = MovieBudget_Prepared['Release Date'].str[-4:].astype(int)
MovieBudget_Prepared['Release Date'].value_counts().sort_values(axis=0).head(50)

1920     1
1935     1
1929     1
1915     1
1931     1
1934     1
1958     1
1941     1
1937     1
1927     1
1947     1
1930     1
1955     2
1938     2
1942     2
1916     2
1944     2
1943     2
1948     2
1950     2
1925     2
1949     2
1936     3
1939     3
1945     3
1933     3
1952     3
1957     3
1954     3
2020     3
1940     4
1959     4
1946     5
1961     5
1960     5
1951     6
1953     6
1966     6
1965     6
1972     7
1964     7
1967     8
1963     8
1973     8
1975     8
1962     8
1968     9
1971     9
1969    10
1956    10
Name: Release Date, dtype: int64

Looking at our year values, we determined that many of the films before 1990 would act as outliers in our analysis. Ultimately, we determined that our stakeholders are seeking analysis on more current trends in the film indusry, so we eliminated the films from before 1990 from our dataset.

In [11]:
# Removing films released before 1990
MovieBudget_Prepared = MovieBudget_Prepared[(MovieBudget_Prepared['Release Date']>1990)].reset_index(drop=True)
MovieBudget_Prepared

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892
3,4,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841
4,5,2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696
...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000
5176,79,1999,Following,6000,48482,240495,234495,40.082500
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000


Finally, for this section, we wanted to get a sense for the spread of the 'Production Budget' distribution. We ran a .describe() method on the dataframe and found the following values, which we will use later in our analysis.

In [12]:
MovieBudget_Prepared['Production Budget'].describe()

count    5.180000e+03
mean     3.394279e+07
std      4.336466e+07
min      1.100000e+03
25%      5.575000e+06
50%      2.000000e+07
75%      4.500000e+07
max      4.250000e+08
Name: Production Budget, dtype: float64

Now our dataset from The Numbers is cleaned and prepared to merge with our IMDb dataset after we analyzed the category options for movie types.

#### 2. Exploring Categorization Options for Films and Merging our Datasets.
Within the IMDb dataset, we identified the 'genres' column contained in the 'movie_basics' table and the names of all the directors associated with each film in the 'directors column, which we will integrate later. 

We first connected to the 'movie_basics' table and applied our data cleaning functions to standardize the 'genres' column like we did before for The Numbers dataset. Then we reconfigured the dataset as a pandas DataFrame, allowing us to more easily manipulate and merge the data with our exisiting dataset. Column headers were changed in order to match our exisiting dataset's headers, allowing us to join our datasets on this shared header appropriately.

In [13]:
# Connecting to Movie Basics Table where genres reside.
movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""
movie_basics_df = pd.read_sql(movie_basics_query, conn)

# Applying functions to clean the column headers and to cast the 'genres' column as a list separated by commas.
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df

# Configuring the SQL table as a pandas DataFrame.
movie_genres_df = pd.DataFrame([movie_basics_df['Movie Id'], movie_basics_df['Primary Title'], movie_basics_df['Genres']])
movie_genres_df = movie_genres_df.transpose()

# Replacing the column header 'Primary Title' with 'Movie' to match our existing dataset column containing the same info.
movie_genres_df.columns = movie_genres_df.columns.str.replace('Primary Title', 'Movie')
movie_genres_df = movie_genres_df.drop_duplicates('Movie')

# Displaying the newly created dataframe containing the 'Movie' title and its corresponding 'Genre(s)'
movie_genres_df

,Movie Id,Movie,Genres
0,tt0063540,Sunghursh,"[Action, Crime, Drama]"
1,tt0066787,One Day Before the Rainy Season,"[Biography, Drama]"
2,tt0069049,The Other Side of the Wind,[Drama]
3,tt0069204,Sabse Bada Sukh,"[Comedy, Drama]"
4,tt0100275,The Wandering Soap Opera,"[Comedy, Drama, Fantasy]"
...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,[Drama]
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,[Documentary]
146141,tt9916706,Dankyavar Danka,[Comedy]
146142,tt9916730,6 Gunn,None


The IMDb data contains many more movies than we have financial information for in The Numbers dataset. We determined that we would only analyze films with sufficient finicial data.

When we merged the datasets, we were left with ONLY the movies that had financial data that we could analyze. 

In [14]:
# Merging The Numbers dataset with the IMDb dataset on the column 'Movie'
Merged_df = MovieBudget_Prepared.merge(movie_genres_df, on = 'Movie', how='left')
Merged_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,[Horror]
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,"[Action, Adventure, Fantasy]"
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,"[Action, Adventure, Sci-Fi]"
3,4,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841,tt2395427,"[Action, Adventure, Sci-Fi]"
4,5,2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,"[Horror, Sci-Fi, Thriller]"
5176,79,1999,Following,6000,48482,240495,234495,40.082500,NaN,NaN
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600,NaN,NaN
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,"[Drama, Horror, Thriller]"


Because the 'Genres' column cateroizes some of the films in multiple genres and also contains null values, further data cleaning was required.

- First we exploded the 'Genres' column, separating each unique genre for each film and creating new records to show each film in each of its respective genres.


- We determined that a film falling into multiple genres needs to be counted for each respective genre that it belongs to, as the film can be duplicated along the genre categorization, without duplicating any of the financial metrics. This is because we do not combine genres via addition or any other equation throughout the rest of our analysis. 


- Then we dropped null values, as we are not able to analyze the films that are missing this key data point.

In [15]:
# Further explanation of dropped values.
MergedExploded_df1 = Merged_df.explode('Genres')
MergedExploded_df1

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600,NaN,NaN
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Thriller


In [16]:
# Explode Genres column to split any movies that are in multiple Genres.
MergedExploded_df = Merged_df.explode('Genres')

# Drop any movies that have a null value in the Genres column, because we cannot run our
#     analysis without a Genre designation.
MergedExploded_df.dropna(subset=['Genres'], inplace=True)

# Displaying our results to check if we were successful.
MergedExploded_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Sci-Fi
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Thriller
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror


Now, with this cleaned and prepared dataset, we are able to move forward and strategize how to best present this data to our stakeholders.

#### 3. Strategizing how to Present our Findings to the Stakeholders.

We explored which film genre on average produces the highest return on investment by various methods that grouped the genre categories and then calulated the mean ROI values for each genre.

We determined that the range of budgets within the 'Production Budget' column was too wide to compare these film genres accurately, so we decided to create 4 distinct bins defined by the quartiles of the 'Production Budget' column. These bins represent 4 investment ranges that can be considered more risky as the amount of funds invested increases. 

Our analysis determined our 4 risk level production budget investment ranges to be based off the quartiles of the 'Production Budget' column:
- Level 1: 0 - 5,575,000
- Level 2: 5,575,000 - 20,000,000
- Level 3: 20,000,000 - 45,000,000
- Level 4: 45,000,000+

In [17]:
# First we sorted the ROI to see which individual films had the highest ROIS.
MovieBudget_NoBin = MergedExploded_df.copy()
MovieBudget_NoBin.sort_values('ROI', ascending=False, inplace=True)
#MovieBudget_NoBin.reset_index(drop=True, inplace=True)
MovieBudget_NoBin

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Horror
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Mystery
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Thriller
4676,12,2012,The Devil Inside,1000000,53262945,101759490,100759490,101.759490,tt1560985,Horror
4897,60,2009,Home,500000,15433,44793168,44293168,89.586336,tt2075392,Drama
...,...,...,...,...,...,...,...,...,...,...
4036,93,2015,Let's Kill Ward's Wife,5000000,0,0,-5000000,0.000000,tt2980708,Crime
4039,96,2012,Should've Been Romeo,5000000,0,0,-5000000,0.000000,tt1717210,Comedy
4039,96,2012,Should've Been Romeo,5000000,0,0,-5000000,0.000000,tt1717210,Drama
4040,97,2015,Aztec Warrior,5000000,0,0,-5000000,0.000000,tt2237914,Action


We ran some exploratory analysis to see how many films are in each genre.

In [18]:
MergedExploded_df['Genres'].value_counts()

Drama          1104
Comedy          646
Action          538
Adventure       413
Thriller        378
Crime           301
Horror          267
Romance         258
Documentary     176
Biography       173
Sci-Fi          172
Mystery         165
Fantasy         148
Family          128
Animation       119
Music            65
History          62
Sport            49
War              30
Western          16
Musical          14
News              1
Name: Genres, dtype: int64

Next, we analyzed each genre's average return on investment, sorted in a descending manner.

In [19]:
MergedExploded_df.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/1103069296.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MergedExploded_df.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)


Genres
Mystery        8.121784
Horror         7.046584
Thriller       4.838324
News           3.894215
Romance        3.524639
Animation      3.502847
Music          3.475146
Biography      3.116844
Comedy         3.039161
Sci-Fi         3.023010
Adventure      2.994670
Fantasy        2.976817
Drama          2.850792
Sport          2.733471
Family         2.655699
Action         2.580998
Documentary    2.401363
Crime          1.929617
History        1.905893
Musical        1.781934
War            1.730887
Western        1.469226
Name: ROI, dtype: float64

The average ROI is pulled down significantly by the many movies that did not make any revenue.

In [20]:
MergedExploded_df.groupby(by='Genres').mean()['Production Budget'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/1644869802.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MergedExploded_df.groupby(by='Genres').mean()['Production Budget'].sort_values(ascending=False)


Genres
Adventure      9.572879e+07
Animation      8.843109e+07
Fantasy        7.666757e+07
Sci-Fi         7.602439e+07
Action         7.168318e+07
Family         5.497129e+07
Western        4.446875e+07
Comedy         3.533861e+07
Musical        3.385000e+07
Crime          2.888937e+07
History        2.853952e+07
Thriller       2.781811e+07
War            2.679333e+07
Drama          2.432458e+07
Biography      2.390655e+07
Mystery        2.296433e+07
Sport          2.292194e+07
Romance        2.092452e+07
Documentary    1.947396e+07
Horror         1.778017e+07
Music          1.617338e+07
News           1.200000e+07
Name: Production Budget, dtype: float64

Because these ROI calculation account for the entire range of production budgets, we felt that presenting this data alone would not be sufficient or fully accurate for our stakeholders. So, we decided to group or film genres by ranges of production budgets or initial investments made/risk taken to make these films by the production companies.

Example 1: the 'Horror' genre has the second highest average ROI, and the third lowest average production budget overall. This would suggest that this genre is relative less risky than almost all of the other genres to produce and it appears to result in almost the highest return on that investment. 

Example 2: Adventure movies are the most expensive to make and their return on investment is ranked 11th in the overall analysis. This means, that Adventure movies require a large initial investment, which a new studio may not have or want to risk paying. 

#### Determing Risk Level Ranges

Recall the spread we found for the initial 'MovieBudget_Prepared' 'Production Budget' column.

This spread includes as many datapoints as we had available to us in The Numbers dataset, therefore offering the most accurate spread of the industry data that we could access. Yes, we ultimately lost some of the film titles as we further cleaned the data, but to use the spread of the 'Production Budget' column after the 'Genres' category was exploded would create duplicate values and ultimately misrepresent our data.

We have provided the spread below from our original calculation.

In [21]:
# Recall the spread we found for the initial 'MovieBudget_Prepared' 'Production Budget' column.
MovieBudget_Prepared['Production Budget'].describe()

count    5.180000e+03
mean     3.394279e+07
std      4.336466e+07
min      1.100000e+03
25%      5.575000e+06
50%      2.000000e+07
75%      4.500000e+07
max      4.250000e+08
Name: Production Budget, dtype: float64

In [22]:
# ORIGINAL: 
# ANOVA
res = smf.ols(formula="ROI ~ C(Genres)", data=MergedExploded_df).fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     3.801
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           1.03e-08
Time:                        13:17:18   Log-Likelihood:                -20055.
No. Observations:                5223   AIC:                         4.015e+04
Df Residuals:                    5201   BIC:                         4.030e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.5810      0.486      5.307      0.000       1.628       3.534
C(Genres)[T.Adventure]       0.4137      0.738      0.561      0.575      -1.033       1.860
C(Genres)[T.Animation]       0.9218      1.143      0.807      0.420      -1.318       3.162
C(Genres)[T.Biography]       0.5358      0.986      0.544      0.587      -1.397       2.469
C(Genres)[T.Comedy]          0.4582      0.658      0.696      0.487      -0.833       1.749
C(Genres)[T.Crime]          -0.6514      0.812     -0.802      0.422      -2.243       0.940
C(Genres)[T.Documentary]    -0.1796      0.979     -0.183      0.854      -2.100       1.741
C(Genres)[T.Drama]           0.2698      0.593      0.455      0.649      -0.893       1.432
C(Genres)[T.Family]          0.0747      1.109      0.067      0.946      -2.100       2.249
C(Genres)[T.Fantasy]         0.3958      1.047      0.378      0.705      -1.657       2.448
C(Genres)[T.History]        -0.6751      1.513     -0.446      0.655      -3.641       2.291
C(Genres)[T.Horror]          4.4656      0.844      5.288      0.000       2.810       6.121
C(Genres)[T.Music]           0.8941      1.481      0.604      0.546      -2.010       3.798
C(Genres)[T.Musical]        -0.7991      3.054     -0.262      0.794      -6.785       5.187
C(Genres)[T.Mystery]         5.5408      1.004      5.520      0.000       3.573       7.509
C(Genres)[T.News]            1.3132     11.290      0.116      0.907     -20.820      23.447
C(Genres)[T.Romance]         0.9436      0.854      1.105      0.269      -0.731       2.618
C(Genres)[T.Sci-Fi]          0.4420      0.988      0.447      0.655      -1.495       2.379
C(Genres)[T.Sport]           0.1525      1.683      0.091      0.928      -3.147       3.452
C(Genres)[T.Thriller]        2.2573      0.757      2.982      0.003       0.773       3.741
C(Genres)[T.War]            -0.8501      2.116     -0.402      0.688      -4.998       3.298
C(Genres)[T.Western]        -1.1118      2.862     -0.389      0.698      -6.722       4.498
==============================================================================
Omnibus:                    13312.143   Durbin-Watson:                   0.594
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        222974994.502
Skew:                          28.402   Prob(JB):                         0.00
Kurtosis:                    1013.622   Cond. No.                         75.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Binning the datasets based off our investment risk levels.

#### Level 1

In [23]:
L1_lower = 0
L1_upper = 5575000

Level_1 = MergedExploded_df[(MergedExploded_df['Production Budget'] <= L1_upper) & (MergedExploded_df['Production Budget'] > L1_lower)]
Level_1

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
3886,17,1996,Shine,5500000,35811509,36672493,31172493,6.667726,tt4064912,Drama
3886,17,1996,Shine,5500000,35811509,36672493,31172493,6.667726,tt4064912,Music
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Comedy
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Drama
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Romance
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Sci-Fi
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Thriller
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror


In [24]:
Level_1['Genres'].value_counts().describe()

count     21.000000
mean      60.714286
std       74.300836
min        3.000000
25%       14.000000
50%       34.000000
75%       77.000000
max      321.000000
Name: Genres, dtype: float64

In [25]:
Level_1_Median = list(Level_1['Genres'].value_counts()[:10].index)
Level_1_Filtered = Level_1.loc[Level_1['Genres'].isin(Level_1_Median)]
Level_1_Filtered['Genres'].value_counts()

Drama          321
Comedy         155
Horror         132
Thriller       121
Romance         78
Documentary     77
Action          65
Mystery         61
Crime           58
Sci-Fi          43
Name: Genres, dtype: int64

In [26]:
Level_1_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/2140362617.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Level_1_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)


Genres
Mystery        16.075685
Horror         10.256561
Thriller        8.648374
Romance         4.508545
Drama           3.248659
Comedy          2.965519
Sci-Fi          2.195603
Documentary     1.847462
Action          1.738472
Crime           1.470922
Name: ROI, dtype: float64

In [27]:
# Linear Regression and Anova
results1 = smf.ols(formula="ROI ~ C(Genres)", data=Level_1_Filtered).fit()
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     3.454
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           0.000326
Time:                        13:17:18   Log-Likelihood:                -5069.8
No. Observations:                1111   AIC:                         1.016e+04
Df Residuals:                    1101   BIC:                         1.021e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    1.7385      2.891      0.601      0.548      -3.934       7.411
C(Genres)[T.Comedy]          1.2270      3.444      0.356      0.722      -5.531       7.986
C(Genres)[T.Crime]          -0.2675      4.210     -0.064      0.949      -8.529       7.994
C(Genres)[T.Documentary]     0.1090      3.926      0.028      0.978      -7.595       7.813
C(Genres)[T.Drama]           1.5102      3.170      0.476      0.634      -4.711       7.731
C(Genres)[T.Horror]          8.5181      3.532      2.412      0.016       1.588      15.448
C(Genres)[T.Mystery]        14.3372      4.155      3.450      0.001       6.184      22.490
C(Genres)[T.Romance]         2.7701      3.915      0.708      0.479      -4.911      10.451
C(Genres)[T.Sci-Fi]          0.4571      4.582      0.100      0.921      -8.533       9.448
C(Genres)[T.Thriller]        6.9099      3.585      1.928      0.054      -0.124      13.943
==============================================================================
Omnibus:                     2163.862   Durbin-Watson:                   0.636
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2889736.441
Skew:                          14.591   Prob(JB):                         0.00
Kurtosis:                     251.139   Cond. No.                         14.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# Filtering Merged_df by L1 Bracket

Level_1 = Merged_df[(Merged_df['Production Budget'] <= L1_upper) & (Merged_df['Production Budget'] > L1_lower)]


# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title'])
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates(subset='Name')

# Creating Top Three Genres Dataframe
Level_1.dropna(inplace=True)
Level_1 = Level_1.merge(joined_df, on = 'Movie', how='left')
TopThreeGenres_df1 = Level_1.loc[(Level_1['Genres'].apply(lambda x: 'Horror' in x)) | (Level_1['Genres'].apply(lambda x: 'Mystery' in x))
             | (Level_1['Genres'].apply(lambda x: 'Thriller' in x))].reset_index(drop=True)
TopThreeGenres_df1.dropna(inplace=True)

TopThreeGenres_df1.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/1291337088.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level_1.dropna(inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/1291337088.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


Name
Chris Lofing          416.564740
William Brent Bell    101.759490
Jordan Peele           51.073590
Bradley Parker         42.411721
Daniel Stamm           38.981056
James DeMonaco         30.422194
Holly Bell             29.242602
Tom Boyle              27.179241
Suzette Reyes          27.179241
Junshu Huang           27.179241
Name: ROI, dtype: float64

#### Level 2

In [29]:
L2_lower = 5575000
L2_upper = 20000000

Level_2 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L2_upper) & (MergedExploded_df['Production Budget'] > L2_lower)]
Level_2

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Drama
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Horror
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Sci-Fi
2604,19,2017,The Shape of Water,19500000,63859435,195790794,176290794,10.040554,tt5580390,Adventure
2604,19,2017,The Shape of Water,19500000,63859435,195790794,176290794,10.040554,tt5580390,Drama
...,...,...,...,...,...,...,...,...,...,...
3874,3,2012,Trade of Innocents,5800000,15091,15091,-5784909,0.002602,tt1772408,Drama
3874,3,2012,Trade of Innocents,5800000,15091,15091,-5784909,0.002602,tt1772408,Thriller
3882,11,2010,Standing Ovation,5600000,531806,531806,-5068194,0.094965,tt1303803,Musical
3883,12,2012,Khiladi 786,5600000,385422,14683763,9083763,2.622101,tt2166214,Action


In [30]:
Level_2['Genres'].value_counts().describe()

count     22.000000
mean      52.818182
std       70.054153
min        1.000000
25%       15.000000
50%       32.000000
75%       66.500000
max      324.000000
Name: Genres, dtype: float64

In [31]:
Level_2_Median = list(Level_2['Genres'].value_counts()[:10].index)
Level_2_Filtered = Level_2.loc[Level_2['Genres'].isin(Level_2_Median)]
Level_2_Filtered['Genres'].value_counts()

Drama        324
Comedy       142
Thriller      90
Crime         79
Romance       74
Horror        67
Biography     65
Action        63
Mystery       38
Adventure     36
Name: Genres, dtype: int64

In [32]:
Level_2_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/3118719430.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Level_2_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)


Genres
Horror       5.106763
Mystery      4.896437
Thriller     3.791583
Romance      3.256027
Biography    3.253414
Comedy       3.068696
Action       3.034745
Drama        2.931449
Adventure    2.409342
Crime        1.799999
Name: ROI, dtype: float64

In [33]:
# Linear Regression and Anova
results2 = smf.ols(formula="ROI ~ C(Genres)", data=Level_2_Filtered).fit()
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2.985
Date:                Thu, 03 Aug 2023   Prob (F-statistic):            0.00163
Time:                        13:17:20   Log-Likelihood:                -2893.2
No. Observations:                 978   AIC:                             5806.
Df Residuals:                     968   BIC:                             5855.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  3.0347      0.590      5.141      0.000       1.876       4.193
C(Genres)[T.Adventure]    -0.6254      0.979     -0.639      0.523      -2.547       1.296
C(Genres)[T.Biography]     0.2187      0.828      0.264      0.792      -1.407       1.844
C(Genres)[T.Comedy]        0.0340      0.709      0.048      0.962      -1.358       1.426
C(Genres)[T.Crime]        -1.2347      0.791     -1.560      0.119      -2.788       0.318
C(Genres)[T.Drama]        -0.1033      0.645     -0.160      0.873      -1.369       1.163
C(Genres)[T.Horror]        2.0720      0.822      2.520      0.012       0.458       3.686
C(Genres)[T.Mystery]       1.8617      0.962      1.934      0.053      -0.027       3.750
C(Genres)[T.Romance]       0.2213      0.803      0.275      0.783      -1.355       1.798
C(Genres)[T.Thriller]      0.7568      0.770      0.983      0.326      -0.754       2.267
==============================================================================
Omnibus:                      730.878   Durbin-Watson:                   0.594
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12465.868
Skew:                           3.322   Prob(JB):                         0.00
Kurtosis:                      19.179   Cond. No.                         14.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
# Filtering Merged_df by L1 Bracket
Level_2 = Merged_df[(Merged_df['Production Budget'] <= L2_upper) & (Merged_df['Production Budget'] > L2_lower)]


# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title'])
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates(subset='Name')

# Creating Top Three Genres Dataframe
Level_2.dropna(inplace=True)
Level_2 = Level_2.merge(joined_df, on = 'Movie', how='left')
TopThreeGenres_df = Level_2.loc[(Level_2['Genres'].apply(lambda x: 'Horror' in x)) | (Level_2['Genres'].apply(lambda x: 'Mystery' in x))
             | (Level_2['Genres'].apply(lambda x: 'Thriller' in x))].reset_index(drop=True)
TopThreeGenres_df.dropna(inplace=True)

TopThreeGenres_df.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/3511462207.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level_2.dropna(inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/3511462207.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


Name
John R. Leonetti     39.517372
Darren Aronofsky     25.482055
James Wan            15.900007
Sohee Myoung          9.227964
Bernard Kordieh       9.227964
Wusi Liu              9.227964
James Watkins         8.597060
The Lowpriest         7.483694
Sebastian Klinger     7.094602
Ari Aster             7.013390
Name: ROI, dtype: float64

<Image>

#### Level 3

In [35]:
L3_lower = 20000000
L3_upper = 45000000

Level_3 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L3_upper) & (MergedExploded_df['Production Budget'] >= L3_lower)]
Level_3

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Biography
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Drama
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Thriller
1312,31,2010,Charlie St. Cloud,44000000,31206263,48478084,4478084,1.101775,tt1438254,Drama
1312,31,2010,Charlie St. Cloud,44000000,31206263,48478084,4478084,1.101775,tt1438254,Fantasy
...,...,...,...,...,...,...,...,...,...,...
2597,12,2016,The History of Love,20000000,0,476624,-19523376,0.023831,tt0443533,Romance
2597,12,2016,The History of Love,20000000,0,476624,-19523376,0.023831,tt0443533,War
2599,14,2014,Dwegons and Leprechauns,20000000,0,0,-20000000,0.000000,tt1134666,Animation
2600,15,2016,Fight Valley,20000000,0,0,-20000000,0.000000,tt4280822,Action


In [36]:
Level_3['Genres'].value_counts().describe()

count     21.000000
mean      60.428571
std       69.047499
min        1.000000
25%       20.000000
50%       36.000000
75%       79.000000
max      290.000000
Name: Genres, dtype: float64

In [37]:
Level_3_Median = list(Level_3['Genres'].value_counts()[:10].index)
Level_3_Filtered = Level_3.loc[Level_3['Genres'].isin(Level_3_Median)]
Level_3_Filtered['Genres'].value_counts()

Drama          290
Comedy         174
Action         133
Crime          103
Thriller        87
Romance         79
Adventure       63
Biography       60
Horror          42
Documentary     39
Name: Genres, dtype: int64

In [38]:
Level_3_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/2816157326.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Level_3_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)


Genres
Horror         3.043820
Romance        3.001602
Documentary    2.939782
Comedy         2.849206
Drama          2.533341
Thriller       2.508820
Biography      2.317956
Adventure      2.250723
Action         2.172090
Crime          2.122229
Name: ROI, dtype: float64

In [39]:
# Linear Regression and Anova
results3 = smf.ols(formula="ROI ~ C(Genres)", data=Level_3_Filtered).fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.183
Date:                Thu, 03 Aug 2023   Prob (F-statistic):              0.302
Time:                        13:17:23   Log-Likelihood:                -2682.3
No. Observations:                1070   AIC:                             5385.
Df Residuals:                    1060   BIC:                             5434.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.1721      0.259      8.401      0.000       1.665       2.679
C(Genres)[T.Adventure]       0.0786      0.456      0.172      0.863      -0.816       0.974
C(Genres)[T.Biography]       0.1459      0.464      0.315      0.753      -0.764       1.056
C(Genres)[T.Comedy]          0.6771      0.343      1.972      0.049       0.003       1.351
C(Genres)[T.Crime]          -0.0499      0.391     -0.127      0.899      -0.818       0.718
C(Genres)[T.Documentary]     0.7677      0.543      1.414      0.158      -0.298       1.833
C(Genres)[T.Drama]           0.3613      0.312      1.157      0.248      -0.251       0.974
C(Genres)[T.Horror]          0.8717      0.528      1.652      0.099      -0.164       1.907
C(Genres)[T.Romance]         0.8295      0.424      1.958      0.050      -0.002       1.661
C(Genres)[T.Thriller]        0.3367      0.411      0.819      0.413      -0.470       1.144
==============================================================================
Omnibus:                      888.254   Durbin-Watson:                   0.559
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23377.774
Skew:                           3.717   Prob(JB):                         0.00
Kurtosis:                      24.659   Cond. No.                         10.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# Filtering Merged_df by L1 Bracket
Level_3 = Merged_df[(Merged_df['Production Budget'] <= L3_upper) & (Merged_df['Production Budget'] > L3_lower)]


# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title'])
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates(subset='Name')

# Creating Top Three Genres Dataframe
Level_3.dropna(inplace=True)
Level_3 = Level_3.merge(joined_df, on = 'Movie', how='left')
TopThreeGenres_df = Level_3.loc[(Level_2['Genres'].apply(lambda x: 'Horror' in x)) | (Level_3['Genres'].apply(lambda x: 'Mystery' in x))
             | (Level_3['Genres'].apply(lambda x: 'Thriller' in x))].reset_index(drop=True)
TopThreeGenres_df.dropna(inplace=True)

TopThreeGenres_df.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/4125452918.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level_3.dropna(inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/4125452918.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


Name
Andy Muschietti       19.927371
Melinte Reitzema      11.437694
Elisa Chee            11.437694
Wes Ball              10.244702
Olivier Megaton        8.395720
Gareth Van Niekerk     5.969123
Vrinda Samartha        5.761664
Braxton Merzlock       5.761664
Neil Burger            5.761664
Dennis Widmyer         5.214340
Name: ROI, dtype: float64

#### Level 4

In [41]:
L4_lower = 45000000
L4_upper = 1000000000

Level_4 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L4_upper) & (MergedExploded_df['Production Budget'] >= L4_lower)]
Level_4

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ROI,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
1306,24,2002,Pinocchio,45000000,3681811,31681811,-13318189,0.704040,tt1488589,Fantasy
1306,24,2002,Pinocchio,45000000,3681811,31681811,-13318189,0.704040,tt1488589,Musical
1308,26,2012,Foodfight!,45000000,0,73706,-44926294,0.001638,tt0249516,Action
1308,26,2012,Foodfight!,45000000,0,73706,-44926294,0.001638,tt0249516,Animation


In [42]:
Level_4['Genres'].value_counts().describe()

count     21.000000
mean      72.238095
std       84.402550
min        4.000000
25%       12.000000
50%       30.000000
75%       82.000000
max      280.000000
Name: Genres, dtype: float64

In [43]:
Level_4_Median = list(Level_4['Genres'].value_counts()[:10].index)
Level_4_Filtered = Level_4.loc[Level_4['Genres'].isin(Level_4_Median)]
Level_4_Filtered['Genres'].value_counts()

Adventure    280
Action       277
Comedy       175
Drama        169
Animation     90
Fantasy       82
Thriller      80
Sci-Fi        79
Crime         61
Family        52
Name: Genres, dtype: int64

In [44]:
Level_4_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/1635439936.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Level_4_Filtered.groupby(by='Genres').mean()['ROI'].sort_values(ascending=False)


Genres
Animation    3.881066
Sci-Fi       3.578737
Adventure    3.338000
Comedy       3.269292
Action       2.871839
Thriller     2.786542
Family       2.586350
Fantasy      2.560044
Drama        2.485187
Crime        2.208390
Name: ROI, dtype: float64

In [45]:
# Linear Regression and Anova
results4 = smf.ols(formula="ROI ~ C(Genres)", data=Level_4_Filtered).fit()
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ROI   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     5.494
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           1.91e-07
Time:                        13:17:25   Log-Likelihood:                -3011.0
No. Observations:                1345   AIC:                             6042.
Df Residuals:                    1335   BIC:                             6094.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  2.8718      0.137     20.979      0.000       2.603       3.140
C(Genres)[T.Adventure]     0.4662      0.193      2.414      0.016       0.087       0.845
C(Genres)[T.Animation]     1.0092      0.276      3.651      0.000       0.467       1.552
C(Genres)[T.Comedy]        0.3975      0.220      1.807      0.071      -0.034       0.829
C(Genres)[T.Crime]        -0.6634      0.322     -2.059      0.040      -1.296      -0.031
C(Genres)[T.Drama]        -0.3867      0.222     -1.739      0.082      -0.823       0.050
C(Genres)[T.Family]       -0.2855      0.344     -0.829      0.407      -0.961       0.390
C(Genres)[T.Fantasy]      -0.3118      0.286     -1.089      0.277      -0.874       0.250
C(Genres)[T.Sci-Fi]        0.7069      0.291      2.433      0.015       0.137       1.277
C(Genres)[T.Thriller]     -0.0853      0.289     -0.295      0.768      -0.653       0.482
==============================================================================
Omnibus:                      566.410   Durbin-Watson:                   0.547
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2791.132
Skew:                           1.946   Prob(JB):                         0.00
Kurtosis:                       8.886   Cond. No.                         8.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
# Filtering Merged_df by L1 Bracket
Level_4 = Merged_df[(Merged_df['Production Budget'] <= L4_upper) & (Merged_df['Production Budget'] > L4_lower)]


# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title'])
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates(subset='Name')

# Creating Top Three Genres Dataframe
Level_4.dropna(inplace=True)
Level_4 = Level_4.merge(joined_df, on = 'Movie', how='left')
TopThreeGenres_df4 = Level_4.loc[(Level_4['Genres'].apply(lambda x: 'Horror' in x)) | (Level_4['Genres'].apply(lambda x: 'Mystery' in x))
             | (Level_4['Genres'].apply(lambda x: 'Thriller' in x))].reset_index(drop=True)
TopThreeGenres_df4.dropna(inplace=True)

TopThreeGenres_df4.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/3749832667.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level_4.dropna(inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_11886/3749832667.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df4.groupby(by='Name').mean()['ROI'].sort_values(ascending=False)[:10]


Name
Ruben Fleischer       7.358867
Gary Breslin          7.358867
Alfonso Cuarón        6.306352
Sam Mendes            5.552635
Paul W.S. Anderson    5.145638
Justin Lin            5.041308
Jay Border            4.570256
Laura Clarke          4.443532
Nan Feix              4.443532
Mickey Fonseca        4.410000
Name: ROI, dtype: float64

In [47]:
# Exporting Dataframes
# Top Directors  
TopThreeGenres_df1.to_csv('/../NewCSVs')
TopThreeGenres_df2.to_csv('/../NewCSVs')
TopThreeGenres_df3.to_csv('/../NewCSVs')
TopThreeGenres_df4.to_csv('/../NewCSVs')

# Top Performing Genre
Level_1_Filtered.to_csv('/../NewCSVs')
Level_2_Filtered.to_csv('/../NewCSVs')
Level_3_Filtered.to_csv('/../NewCSVs')
Level_4_Filtered.to_csv('/../NewCSVs')

NameError: name 'TopThreeGenres_df1' is not defined